In [ ]:
# 불완전한 전처리

# 기본 import
import numpy as np
import pandas as pd
import  matplotlib.pyplot as plt

# konlpy 인스톨, import
! pip install kss konlpy
import nltk, kss, konlpy

# t_data2  [ kobill 문자열데이터 불러오기 ]
from konlpy.corpus import kobill
kobill.fileids() # kobill 파일이름 확인
t_data2 = kobill.open('1809890.txt').read() # 파일 리드

# t_data2_tk  [ t_data2 를 명사 기준으로 토크나이즈 ]  
from konlpy.tag import Okt # 한글 형태소 태그 토크나이저
tk = Okt()
t_data2_tk = tk.nouns(t_data2) # 명사만
t_data2_tk = [i for i in t_data2_tk if len(i)>1]  # 단어길이 2이상만 살린다

# t_data2_tk 를 빈도순으로 1부터 오름차순 정렬한다
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(t_data2_tk)
tk.index_word # 빈도순 정렬 : 1이 가장 높은 빈도
tk.texts_to_sequences(t_data2_tk) # 정수화 (단어가 묶음이 아니라 하나 하나씩 정수화된 상태)

In [ ]:
# 문자열데이터로드 > 문장토크나이즈 > 단어토크나이즈 > 정수인코딩 > 패딩

# 기본 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 텍스트 다운로드, import
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt -O 2016-10-20.txt
!pip install soynlp
from soynlp import DoublespaceLineCorpus
 
# t_data  [ kolaw 문자열데이터 불러오기 ]
from konlpy.corpus import kolaw 
kolaw.fileids() # kolaw 파일이름 확인
t_data = kolaw.open('constitution.txt').read() # 파일 리드 / 리드로 불러오면 str 타입으로 불러온다
t_data = kolaw.open('constitution.txt') # 이렇게 불러와야 아래 더블스페이스에 쓸 수 있다

# t_data_tk  [ t_data 를 문장 토크나이즈 ]     
t_data_tk = DoublespaceLineCorpus(t_data.name,iter_sent=True) #이터센트트루 들어가면 문장토큰화 ( 생략시 문서로쪼개기에서 > 문장으로쪼개기의 차이)  # 모양 확인법 list(t_data_tk) 

# t_data_tk_tk  [ t_data_tk 를 단어 토크나이즈 ]
from konlpy.tag import Okt
tk = Okt()
t_data_tk_tk = [] 
for i in t_data_tk:
  t_data_tk2 = tk.nouns(i)
  t_data_tk2 = [i for i in t_data_tk2 if len(i)>1] # 단어길이 2이상만 살림
  t_data_tk_tk.append(t_data_tk2) # d 로 넣어버림

# t_data_tk_tk_num [ t_data_tk_tk 를 정수화 ]
tk = Tokenizer()
tk.fit_on_texts(t_data_tk_tk)
tk.index_word # 빈도순 1부터 오름차순 정렬
t_data_tk_tk_num = tk.texts_to_sequences(t_data_tk_tk)  # d 를 정수화

# t_data_tk_tk_num_pad [ t_data_tk_tk_num 을 패딩 ] 
from tensorflow.keras.preprocessing.sequence import pad_sequences
t_data_tk_tk_num_pad = pad_sequences(t_data_tk_tk_num, padding='post', truncating='post')

In [ ]:
# 문자열데이터 > 명사단어토크나이즈 > 텍스트화 > 그래프, 워드클라우드 시각화

# 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!apt-get update -qq
!apt-get install fonts-nanum* -qq

# t_data 리드
from konlpy.corpus import kolaw
t_data = kolaw.open('constitution.txt').read()

# t_data_tk  [ t_data 를 명사 단어 토크나이즈 ]
from konlpy.tag import Okt # 단어토크나이즈
tk = Okt()
t_data_tk = tk.nouns(t_data) # 명사로 토크나이즈

# t_data_tk_txt [ t_data_tk 를 텍스트화 ]
from nltk import Text
t_data_tk_txt = Text(t_data_tk,name="kolaw")
t_data_tk_txt

# 그래프 시각화
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')
t_data_tk_txt.plot(15)
plt.show()

# 워드클라우드 시각화
from wordcloud import WordCloud
wc=WordCloud(font_path='NanumGothic',width=1000,height=600,background_color='white')
plt.imshow(wc.generate_from_frequencies(t_data_tk_txt.vocab()))
plt.axis('off')

In [ ]:
# 데이터 > 단어토크나이즈 > 엔그램

# 설치, import
import nltk
from nltk import bigrams,word_tokenize
from nltk.util import ngrams
nltk.download('movie_reviews')
nltk.download('punkt')

# 데이터 > 단어토크나이즈 > 엔그램(2) > fd로 확률 계산
from nltk import ConditionalFreqDist
t = 'l am a boy'
t2 = 'you are a boy'
t_tk = word_tokenize(t)  # t를 단어토크나이즈
t_tk_ng = ngrams(t_tk,2,pad_left=True,pad_right=True,left_pad_symbol='SS',right_pad_symbol='SE') # tk를 엔그렘(2) # 모양확인 list(ng_d)
t_tk_ng_fd = ConditionalFreqDist([(i[0],i[1])for i in t_tk_ng]) # 앞단어:뒷단어가뭐가나올까의 확률 1:100%  # i[0],i[1] 바이그램상태에서 앞단어-뒷단어  # fd['SS'] 라고 치면 'SS'다음 나올 문자 : 확률 / fd 출력하면 전체

# 단어토크나이즈되어있는데이터 > 엔그램(2)
문장 = movie_reviews.sents()[0]   # movie_reviews.sents() 는 단어토큰화까지 끝난 데이터 여기 [0]을 하면 단어들이 묶인 리스트 하나
list(ngrams(문장,2,pad_left=True,pad_right=True,left_pad_symbol='SS',right_pad_symbol='SE')) # 엔그램(2) 후 바로 시각화 # 엔그램모양확인 list()


In [57]:
from nltk import ConditionalFreqDist  # 앞단어:뒷단어가뭐가나올까의 확률 1:100%  # i[0],i[1] 바이그램상태에서 앞단어-뒷단어  # fd['SS'] 라고 치면 'SS'다음 나올 문자 : 확률 / fd 출력하면 전체
from nltk.corpus import movie_reviews
data_l=[]
for 문장 in movie_reviews.sents():   # 단어토크나이즈끝난데이터, 단어들의묶음으로 되있음  즉 문장한덩어리가 들어감
  bg_d = ngrams(문장,2,pad_left=True,pad_right=True,left_pad_symbol='SS',right_pad_symbol='SE')
  data_l += [토큰_data for 토큰_data in bg_d]
cfd = ConditionalFreqDist(data_l)

In [58]:
movie_reviews.sents()

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]

In [41]:
len(cfd.conditions())

39769

In [42]:
cfd['SS'].most_common(10)

[('the', 7965),
 ('it', 3038),
 ('i', 2350),
 ('but', 1754),
 ('he', 1642),
 ('in', 1629),
 ('this', 1625),
 ('and', 1607),
 ('there', 1280),
 ('.', 1230)]

In [43]:
cfd['the'].most_common(10)

[('film', 4542),
 ('movie', 2147),
 ('story', 985),
 ('most', 945),
 ('first', 902),
 ('same', 774),
 ('only', 665),
 ('end', 664),
 ('best', 642),
 ('audience', 620)]

In [44]:
cfd['movie'].most_common(10)

[('.', 802),
 (',', 659),
 ('is', 469),
 ("'", 234),
 ('that', 210),
 ('was', 123),
 ('and', 105),
 ('with', 87),
 ('to', 72),
 ('has', 69)]

In [45]:
cfd['.'].most_common(10)

[('SE', 57626),
 ('.', 1893),
 ('"', 1854),
 (')', 535),
 ('s', 129),
 ('i', 119),
 ('a', 118),
 ('and', 116),
 ('?', 108),
 ('the', 98)]

In [46]:
from nltk.probability import ConditionalProbDist,MLEProbDist

In [47]:
cpd=ConditionalProbDist(cfd,MLEProbDist)

In [48]:
cpd['the'].prob('movie')

0.0280547243528597

In [49]:
cpd['movie'].prob('.')

0.13897071564720154

In [50]:
cpd['.'].prob('the')

0.0014876434513328071

P(SS<문장>SE)=P(시작단어|SS)*P(|)*P(|)*P(|)*P(|)*P(|)*P(SE|종료단어)

In [ ]:
import numpy as np
def s_sc_f(x):#바이그램 언어 모형의 확률 연산 함수
  p=0.0
  for i in range(len(x)-1):
    c=x[i]
    w=x[i+1]
    p+=np.log(cpd[c].prob(w)+np.finfo(float).eps)
  return np.exp(p)
t_data=['the','movie','.']
t_data2=['movie','.','the']
s_sc_f(t_data),s_sc_f(t_data2)

In [ ]:
from nltk.util import ngrams#앞단어 뒷단어 data 생성용
from nltk import ConditionalFreqDist# 문맥별 단어 빈도수 
from nltk.probability import ConditionalProbDist #조건부 확률 
from nltk.probability import MLEProbDist #최대 우도 추정 값 계산

In [ ]:
nltk.download('movie_reviews')
nltk.download('punkt')

데이터 수집

In [ ]:
from nltk.corpus import movie_reviews
data=movie_reviews.sents()

데이터 전처리<br>
  -토큰화<br>
  -정형화<br>
  -정규화<br>

In [ ]:
data_l=[]
for 문장 in data:
  bg=ngrams(문장, 2, pad_left=True,left_pad_symbol='SS',pad_right=True,right_pad_symbol='SE')
  data_l+=[t for t in bg]

모델 학습

In [ ]:
cfd = ConditionalFreqDist(data_l)
cpd = ConditionalProbDist(cfd,MLEProbDist)

모델 검증(생략)

생성 <자동 랜덤 문장 생성기>

In [ ]:
import random
random.seed(10)
cpd['SS'].generate()

In [ ]:
cpd['when'].generate()

In [ ]:
cpd['i'].generate()

In [ ]:
cpd['still'].generate()

In [ ]:
cpd['know'].generate()

In [ ]:
st='SS'
all_l=[]
import random
random.seed(10)
while True:##SS<문장>SE
  st=cpd[st].generate()
  all_l.append(st)
  if st=='SE':
    all_l.pop()
    break
생성된_data=' '.join(all_l)
생성된_data


In [ ]:
import random
def 정리_생성(c='SS',s_n=10):
  random.seed(s_n)
  all_data=[]
  while True:
    if c not in cpd:
      break
    w = cpd[c].generate()
    if w=='SE':
      break
    elif w in ['i','ii','iii']:
      w2=w.upper()
    elif w in ["mr", "luc", "i", "robin", "williams", "cindy", "crawford"]:
      w2=w.title()
    else:
      w2=w
    
    if c=="SS":
      all_data.append(w2.title())
    elif c in ["'","\"",'(']:
      all_data.append(w2)
    elif w in ["'",",",".",")",":",";","?"]:
      all_data.append(w2)
    else:
      all_data.append(" "+w2)
    
    c = w
  if len(all_data)>0:
    return ''.join(all_data)
  return "문장을 생성 할 수 없습니다."

In [ ]:
정리_생성()

In [ ]:
정리_생성('gdf')

In [ ]:
정리_생성('k',100)